# Verification of references to UK Catalysis Hub 
A list of articles is obtainded from publish or perish. This list will contain a titles and some IDs whic need to be verified. 

The criteria for adding a publication to the database are: 
a) has an explicit acknowledgement of UK Catalysis Hub
b) mentions one of the UK Catalysis Hub grants
c) has two or more authors with affiliation to UK Catalysis Hub
d) acknowledges support from a scientist affiliated to UK Catalysis Hub.

In [1]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
# library for getting data from crossref
import lib.crossref_api as cr_api
#library for handling json files
import json


In [21]:
def get_cr_json_object(cr_doi):
  crjd = None
  doi_file = 'json_files/' + cr_doi.replace('/','_').lower() + '.json'
  if not Path(doi_file).is_file():
    crjd = cr_api.getBibData(cr_doi)
    with open(doi_file, 'w', encoding='utf-8') as f:
                json.dump(crjd, f, ensure_ascii=False, indent=4)
  else:
    jf = open(doi_file, 'r')
    crjd = json.load(jf)
  return crjd, doi_file

def get_titles(str_pub_title, db_name = "prev_search.sqlite3"):
    print(db_name)
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'prev_pop_searches'
    fields_required = "Num, Title"
    filter_str = "Title like '"+str_pub_title[0]+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

def get_titles_and_dois(str_pub_title, db_name = "app_db.sqlite3"):
    print(db_name)
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, title, doi"
    filter_str = "Title like '"+str_pub_title[0]+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

Get the name of the file with the results of the PoP search:

In [4]:
# input file with path: pop_searches/PoPCites20201017.csv
new_results_file = ""
while not Path(new_results_file).is_file():
    print('Please enter the name of the input file:')
    new_results_file = input()

Please enter the name of the input file:
pop_searches/PoPCites20201017.csv


Get the name of the db file with previous results of the PoP search:

In [5]:
# previous results db file with path: db_files/prev_search.sqlite3

previous_db = ""
while not Path(previous_db).is_file():
    print('Please enter the name of the previous results file:')
    previous_db = input()

Please enter the name of the previous results file:
db_files/prev_search.sqlite3


Get the name of the current app db file:

In [6]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = ""
while not Path(ukchapp_db).is_file():
    print('Please enter the name of the previous results file:')
    ukchapp_db = input()

Please enter the name of the previous results file:
db_files/app_db.sqlite3


Set the name of the output file


In [7]:
nr_wf = new_results_file[:-4]+"_wf.csv"
print("Verifying if the articles listed in: \n\t", Path(new_results_file).name)
print("where included in previous searches: \n\t", Path(previous_db).name)

print("The results will bt saved in: \n\t", nr_wf)

Verifying if the articles listed in: 
	 PoPCites20201017.csv
where included in previous searches: 
	 prev_search.sqlite3
The results will bt saved in: 
	 pop_searches/PoPCites20201017_wf.csv


In [8]:
working_file = wf_fields = None
current_pass = 0
if Path(nr_wf).is_file():
    working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
    for art_num in tqdm_notebook(working_file):
        if current_pass < int(working_file[art_num]['ignore']):
            current_pass = int(working_file[art_num]['ignore'])
print("Current pass:", current_pass)


Current pass: 7


## Verify if already processed titles are included
Read data and verify if results in file have already been included in previous searches


In [9]:
if current_pass == 0:
    current_initial = ""
    db_titles = []
    for art_num in tqdm_notebook(working_file):
        new_title = working_file[art_num]['Title'].lower()
        working_file[art_num]['ignore'] = 0 
        working_file[art_num]['previous'] = 0 
        working_file[art_num]['similarity'] = 0.0
        if current_initial == "" or current_initial != new_title[0]:
            print("new intital ", new_title[0])
            current_initial = new_title[0]
            db_titles = get_titles(current_initial, previous_db)
            
        for prev_pair in db_titles:
            prev_num = prev_pair[0]
            used_title = prev_pair[1].lower()
            # if titles match exactly or simialarity > 0.8 ignore
            title_similarity = txtc.similar(new_title, used_title)
            if title_similarity > 0.80:
                #print(art_num, 'Title:', new_title, "already processed", prev_num, used_title)
                working_file[art_num]['ignore'] = 1
                working_file[art_num]['previous'] = prev_num
                working_file[art_num]['similarity'] = title_similarity
                break

    csvh.write_csv_data(working_file, nr_wf)  
    print(nr_wf)

## Check Titles in app
Verify if the title is in the app

In [10]:
# verify that titles are not in the app_db (if they are  also get DOI)
if current_pass == 1: 
    db_titles = []
    current_initial = ""
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0':
            new_title = working_file[art_num]['Title'].lower()
            if current_initial == "" or current_initial != new_title[0]:
                print("new intital ", new_title[0])
                current_initial = new_title[0]
                db_titles = get_titles_and_dois(current_initial, ukchapp_db)
            for art_in_db in db_titles:
                prev_num = art_in_db[0]
                used_title = art_in_db[1].lower()
                # if titles match exactly or simialarity > 0.8 ignore
                title_similarity = txtc.similar(new_title, used_title)
                if title_similarity > 0.80:
                    #print(art_num, 'Title:', new_title, "already processed", prev_num, used_title)
                    working_file[art_num]['ignore'] = 2
                    working_file[art_num]['previous'] = prev_num
                    working_file[art_num]['similarity'] = title_similarity
                    working_file[art_num]['DOIcr'] = art_in_db[2]
                    break                
    csvh.write_csv_data(working_file, nr_wf)

## Check Title Wording
Using the workds in previous catalysis hub papers check if the title is likely to be a cat hub title

In [11]:
if current_pass < 3:
    # pass 2
    # check titles for likelihood of being catalysis articles using keywords from titles in current DB 
    print("Get word list from DB")
    x = dbh.DataBaseAdapter(ukchapp_db)
    db_titles = x.get_value_list('articles','title')
    title_words = set()
    ignore_words=set(['the','of','to','and','a','in','is','it', 'their', 'so', 'as'])
    average = 0
    words_sum = 0.0
    for title in db_titles:
        one_title = set(title.lower().split())
        one_title = one_title - ignore_words
        title_words = title_words.union(one_title)
        words_sum += len(one_title) 
        
    average = words_sum /len(db_titles)
    print("Average words per title:", average)
    title_words = title_words - ignore_words
    for art_num in tqdm_notebook(working_file):
        if 0 == int(working_file[art_num]['ignore']):
            art_title = working_file[art_num]['Title']
            art_words = set(art_title.lower().split())
            occurrences = len(title_words.intersection(art_words))
            working_file[art_num]['keywords']=occurrences
            if occurrences == 0:
                print("occurrences:", occurrences, "in title:", art_title)
                working_file[art_num]['ignore']=3
            else:
                print("occurrences:", occurrences, "in title:", art_title)
    csvh.write_csv_data(working_file, nr_wf)
    x.close()
    current_pass = 3

In [12]:
if current_pass == 3:
    i = 0
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0':
            inspected = False
            while not inspected:
                new_title = working_file[art_num]['Title']
                keywords = working_file[art_num]['keywords']
                #print (keywords, new_title)
                if keywords < 3 and not ("cataly" in new_title.lower()):
                # ignore  it because it does not contains cataly in title
                    working_file[art_num]['ignore']=4 # visual inspection
                    inspected = True
                else:
                    inspected = True
    print("To Process:", i, "Pass:", current_pass)
    csvh.write_csv_data(working_file, nr_wf)
    current_pass = 4

## Get DOIs for Articles
The remaining titles need to be further analysed. Recovering their DOIs helps to obtain abstracts and acknowledgement statements. 

In [13]:
working_file = wf_fields = None
current_pass = 0
if Path(nr_wf).is_file():
    working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
    for art_num in tqdm_notebook(working_file):
        if current_pass < int(working_file[art_num]['ignore']):
            current_pass = int(working_file[art_num]['ignore'])
print("Current pass:", current_pass)
if current_pass == 4:
    i = 0
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0' and working_file[art_num]['DOIcr']=="":
            new_title = working_file[art_num]['Title']
            new_doi = cr_api.getDOIForTitle(new_title)
            if new_doi == "":
                #print("Missing DOI:", new_title)
                working_file[art_num]['ignore'] = '5'
                i +=1
            else:
                #print("DOI found:", new_doi, "for:", new_title)
                working_file[art_num]['DOIcr'] = new_doi
                working_file[art_num]['ignore'] = '0'
    print("without DOI:", i)
    csvh.write_csv_data(working_file, nr_wf)
    current_pass = 5


Current pass: 7


## Verify DOIs in DB
Verify that articles do not exist in the DB

In [14]:
working_file = wf_fields = None
current_pass = 0
if Path(nr_wf).is_file():
    working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
    for art_num in tqdm_notebook(working_file):
        if current_pass < int(working_file[art_num]['ignore']):
            current_pass = int(working_file[art_num]['ignore'])
print("Current pass:", current_pass)
if current_pass == 5:
    i = 0
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0':
            new_title = working_file[art_num]['Title']
            new_doi = working_file[art_num]['DOIcr'].strip()
            db_title = db_conn.get_title(new_doi)
            if db_title == None:
                print("Not in DB:", new_doi, new_title)
            else:
                print("Already in DB:", new_doi, "for:", new_title, db_title)
                working_file[art_num]['ignore'] = '6'
    print("without DOI:", i)
    csvh.write_csv_data(working_file, nr_wf)
    current_pass = 6
    dbh.close()


Current pass: 7


## Get full json files for remaining articles 

In [22]:
working_file = wf_fields = None
current_pass = 0
if Path(nr_wf).is_file():
    working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
    for art_num in tqdm_notebook(working_file):
        if current_pass < int(working_file[art_num]['ignore']):
            current_pass = int(working_file[art_num]['ignore'])
            
print("Current pass:", current_pass)

if current_pass >= 6:
    i = 0
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0':
            article_title = working_file[art_num]['Title']
            article_doi = working_file[art_num]['DOIcr']
            article_url =working_file[art_num]['ArticleURL']
            #print("Getting Schema:", article_title, article_doi, article_url)
            # try to retrive html page for article using link from crossref first
            # and if not try url from pop
            # find reference to uk catalysis hub in html text
            # if found mark as relevant
            data, file_name = get_cr_json_object(article_doi)
            if data != {}:
                working_file[art_num]['file'] = file_name
    csvh.write_csv_data(working_file, nr_wf)


Current pass: 7


In [ ]:
#***************************************************************************************************************
# Wait do not run this yet
#***************************************************************************************************************
if current_pass >= 6:
    i = 0
    for art_num in tqdm_notebook(working_file):
        if working_file[art_num]['ignore']=='0':
            article_title = working_file[art_num]['Title']
            article_doi = working_file[art_num]['DOIcr']
            article_url =working_file[art_num]['ArticleURL']
            print("Analysing:", article_title, article_doi, article_url)
            # try to retrive html page for article using link from crossref first
            # and if not try url from pop
            # find reference to uk catalysis hub in html text
            # if found mark as relevant
            found = ""
            referents = ["uk catalysis hub", "uk catalysis", "catalysis hub",
                 'EP/R026645/1', 'resources', 'EP/K014668/1', 'EPSRC', 'EP/K014714/1',
                 'Hub','provided', 'grant', 'biocatalysis', 'EP/R026815/1', 'EP/R026939/1',
                 'support', 'membership', 'EP/M013219/1', 'UK', 'kindly', 'Catalysis',
                 'funded', 'EP/R027129/1', 'Consortium', 'thanked', 'EP/K014854/1', 'EP/K014706/2']
            found = urlh.findFromDOI(article_title, article_doi, referents)
            working_file[art_num]['checked_doi'] = 1
            working_file[art_num]['ack_doi'] = found
            found = urlh.findFromURI(article_title, article_url, referents)
            working_file[art_num]['checked_url'] = 1
            working_file[art_num]['ack_url'] = found
            print("Ack:", found)
    csvh.write_csv_data(working_file, nr_wf)